In [ ]:
import json

import numpy as np
import pandas as pd
import pymongo
import scipy
from bson import ObjectId, json_util
from pandas.io.json import json_normalize
from pymongo import MongoClient as Connection
from scipy import sparse, spatial
from scipy.spatial.distance import cdist, pdist, squareform
from sklearn.metrics.pairwise import cosine_similarity

# scientific notation disabled form smaller numbers
pd.options.display.float_format = '{:.5f}'.format

# alles resultate anzeigen und nicht nur das letzte
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# display multiple outputs in one row
import pandas as pd
import numpy as np
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [ ]:
connection = Connection()
db = connection.recipe_db
input_data = db.recipes_test_copy

data = json.loads(json_util.dumps(input_data.find()))
#norm_data = json_normalize(data)

In [ ]:
ing = json_normalize(data, record_path='ingredients',
                             meta='id', record_prefix='ingredients_', errors='ignore')

ing = pd.DataFrame(ing)

nutritions = json_normalize(data, record_path='nutritions',
                            meta=['id', 'prep_time', 'rating', 'rating_count', 'ready_in_time', 'review_count'])

# ing

In [ ]:
# data cleansing - Ingredients

# schmeiss alle zutaten raus, die weniger als 5 mal verwendet werden
# setze multiinde auf 'id' und 'ingredients_id'
ingredients = ing.set_index(['id', 'ingredients_id'])

# filtere alle Zutaten samit ihrer rezepte id, die weniger gleich 5 mal vorkommen
ingredients_eqles_5_ing = ingredients.groupby(
    'ingredients_id').filter(lambda x: len(x) <= 5)

# droppe alle rezepte, die eine Zutate besitzen, die weniger gleich 5 Mal vorkommen
ingredients_filt = ingredients.drop(ingredients_eqles_5_ing.index.get_level_values('id').values, level=0)

# drop alls rows with ingredients_id == 0
ingredients_eqal_zero = ingredients_filt[ingredients_filt.index.get_level_values('ingredients_id') == 0]
ingredients_filt = ingredients_filt.drop(ingredients_eqal_zero.index.get_level_values('id').values, level=0)

ingredients_filt

In [ ]:
# filtere Rezepte aus den Metadaten anhand ihrerr Zutatenmenge raus.
not_wanted_recipes = recipe_db[recipe_db.mask(recipe_db == 0).count(axis=1) <= 4]

mask = nutritions['id'].isin(not_wanted_recipes.index.values)

nutritions_filt = nutritions.loc[~mask]

In [ ]:
asd[ingredients['id'] == 17739]

In [ ]:
ingredients[ingredients['id'] == 17739]
ingredients[ingredients['ingredients_id'] == 4171]

In [ ]:
id_list = [215014, 8669, 16700, 16354, 12720, 8652, 8887, 51283, 45954, 213742, 14595,
           219164, 16348, 143082, 8665, 11758, 223042, 236609, 8778, 65896, 24264, 11679, 141678, 9023]

recipe_db = pd.get_dummies(ingredients_filt['ingredients_id']).groupby(
    ingredients_filt['id']).apply(max)

In [ ]:
# filtere Rezepte aus den Metadaten anhand ihrerr Zutatenmenge raus.
not_wanted_recipes = recipe_db[recipe_db.mask(
    recipe_db == 0).count(axis=1) <= 4]

mask = nutritions['id'].isin(not_wanted_recipes.index.values)

nutritions_filt = nutritions.loc[~mask]

nutrition_db = nutritions_filt.pivot_table(
    index=['id'],
    columns=['name'],
    values=['amount'],
).reset_index()

nutrition_db.set_index('id', inplace=True)
# remove multiindex 'amount'

nutrition_db.columns = nutrition_db.columns.droplevel(0)

nutrition_db = nutrition_db.dropna()

In [ ]:
nutritions_test_25 = nutrition_db[nutrition_db.index.isin(id_list)]
asd = ingredients_filt.groupby(['id']).sum()
# ingredients_filt
# asd.loc[[17739]]

In [ ]:
recipe_db_filt = recipe_db[recipe_db.mask(recipe_db == 0).count(axis=1) >= 5]

user_nutrition = nutrition_db[nutrition_db.index.isin(id_list)]

user_recipes = recipe_db_filt[recipe_db_filt.index.isin(id_list)]

In [ ]:
ingredients_filt.loc[[17739]]

In [ ]:
# create a function called times100
def portion_to_6(x):
    a = x / 4
    a = a * 2
    x = x + a
    return x

In [ ]:
ingredients_filt['ingredients_grams'] = ingredients_filt['ingredients_grams'].apply(
    portion_to_6)

In [ ]:
asd2 = ingredients_filt.groupby(['id']).sum()
asd2
recipe_summed_weight = asd2.drop(columns='ingredients_id')
recipe_summed_weight.loc[[17739]]

In [ ]:
test = nutrition_db.join(recipe_summed_weight, how='inner')
test

In [ ]:
test.loc[[17739]]

In [ ]:
def capitalizer(x): return x.upper()


# Cholesterol = mg
# Sodium = mg
# Folate = mcg
# Magnesium = mg
# Vitamin B6 = mg
# Niacin Equivalents = mg
# Thiamin = mg
# Iron = mg
# Calcium = mg
# Vitamin C = mg
# Potassium = mg
# Vitamin A - IU = 1IU = 0.6 mcg
test['Cholesterol'] = test['Cholesterol'].apply(lambda x: x / 1000)
test['Sodium'] = test['Sodium'].apply(lambda x: x / 1000)
test['Magnesium'] = test['Magnesium'].apply(lambda x: x / 1000)
test['Vitamin B6'] = test['Vitamin B6'].apply(lambda x: x / 1000)
test['Niacin Equivalents'] = test['Niacin Equivalents'].apply(lambda x: x / 1000)
test['Thiamin'] = test['Thiamin'].apply(lambda x: x / 1000)
test['Iron'] = test['Iron'].apply(lambda x: x / 1000)
test['Calcium'] = test['Calcium'].apply(lambda x: x / 1000)
test['Vitamin C'] = test['Vitamin C'].apply(lambda x: x / 1000)
test['Potassium'] = test['Potassium'].apply(lambda x: x / 1000)

test['Folate'] = test['Folate'].apply(lambda x: x / 1000000)
test['Vitamin A - IU'] = test['Vitamin A - IU'].apply(lambda x: x * 0,6/ 1000000)


In [ ]:
# create a function called times100
def percent_to_1000(x):
    a = x * 100 / x.ingredients_grams
#     if x == x.ingredients_grams:
#         return
#     a = a * 2
#     x = x + a
    return a
#     print(x.ingredients_grams)

percent_test = test
asd = percent_test.apply(percent_to_1000, axis = 1)

In [ ]:
asd['Calories'] = test['Calories']
asd['Calories from Fat'] = test['Calories from Fat']
asd['ingredients_grams'] = test['ingredients_grams']

In [ ]:
asd = asd.sort_values(by='Fat', ascending=False)

In [ ]:
asd